# Exercise 3: ChatGPT-Powered NLP on Customer Support Tickets
### Objective
Use ChatGPT API to classify, analyze, and extract actionable insights from unstructured customer service ticket data. Compare traditional keyword-based methods with AI-driven approaches.

## Step 1: Load the Dataset
Load customer support tickets into a DataFrame to begin processing.

In [1]:
import pandas as pd

# Load the dataset
# Change the path below to your path
df = pd.read_csv('support_tickets.csv')
df.head()

,ticket_id,customer_comment,product,channel
0,1001,The battery life is terrible and the device ov...,Smartphone,Email
1,1002,I ordered the product two weeks ago and still ...,Tablet,Web Form
2,1003,The price dropped right after I bought it. Ver...,Headphones,Chat
3,1004,"Customer service was helpful, but the setup in...",Smartwatch,Phone
4,1005,Love the product! The sound quality is amazing...,Speaker,Email


## Step 2: Keyword-Based Tagging (Baseline)
Apply a simple rule-based function to tag each comment into categories: Billing, Technical, Access, Other.

In [2]:
def categorize_ticket(text):
    text = text.lower()
    if 'bill' in text or 'charge' in text:
        return 'Billing'
    elif 'error' in text or 'bug' in text:
        return 'Technical'
    elif 'login' in text or 'access' in text:
        return 'Access'
    else:
        return 'Other'

df['keyword_category'] = df['customer_comment'].apply(categorize_ticket)
df[['customer_comment', 'keyword_category']].head()

,customer_comment,keyword_category
0,The battery life is terrible and the device ov...,Other
1,I ordered the product two weeks ago and still ...,Other
2,The price dropped right after I bought it. Ver...,Other
3,"Customer service was helpful, but the setup in...",Other
4,Love the product! The sound quality is amazing...,Other


## Step 3: Classification with ChatGPT API
Use OpenAI’s GPT model to classify customer comments, assess urgency, and suggest an action.

In [3]:
import openai
import os

# Set your API key (ensure it's set in your environment)
#os.environ["OPENAI_API_KEY"]= "insert your key here"
KEY=os.getenv("OPENAI_API_KEY")
openai.api_key = KEY 


# Instantiate the OpenAI client
client = openai.OpenAI(api_key=KEY)

def classify_with_chatgpt(comment):
    messages = [
        {"role": "system", "content": "You are a helpful customer support assistant."},
        {"role": "user", "content": f"Classify this support comment: '{comment}' into one of the categories (Billing, Technical, Access, Other). Also state the urgency level (Low, Medium, High) and what action should be taken."}
    ]
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    return response.choices[0].message.content

# Example
classify_with_chatgpt(df['customer_comment'].iloc[0])

'Category: Technical\nUrgency Level: High\nAction to be Taken: This issue seems critical and thus, should be escalated to the technical team immediately for further investigation and resolution. Advising the customer to stop using the device to avoid potential hazards if the device gets overheated. At the same time, ensure the customer that this issue will be resolved as soon as possible by our team.'

## Step 4: Generate Actionable Recommendations
Use ChatGPT to suggest service improvements based on customer feedback.

In [4]:
def get_recommendations(comment):
    messages = [
        {"role": "system", "content": "You are a business analyst generating strategic recommendations based on customer feedback."},
        {"role": "user", "content": f"Based on this customer feedback, what are three specific actions the company could take to improve customer experience? Please explain briefly.\n\nFeedback: {comment}"}
    ]
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    return response.choices[0].message.content

# Example
get_recommendations(df['customer_comment'].iloc[0])

"1. Enhance Battery Life: The company could invest in R&D to improve the battery life of the device. This could involve using more energy-efficient components or enhancing the software to optimize power consumption. Better battery life would extend the usage time of the device, making for a more satisfying customer experience.\n\n2. Improve Cooling Mechanisms: The company could also work on advancing the cooling system within the device to prevent overheating. This could potentially be achieved through better internal architecture, enhanced ventilation, use of heat resistant materials or even integrating a more advanced cooling technology.\n\n3. Provide Guidance on Proper Usage: Given these concerns, creating detailed user guidelines on how to optimize device use could prevent overheating and prolong battery life. This could include tips such as not using certain power-draining applications continuously, not exposing the device to direct sunlight for prolonged periods, or maybe even su

## Step 5: Executive Summary Generation
Summarize multiple customer comments into a high-level report for business stakeholders.

In [5]:
def create_summary(comments):
    joined_comments = "\n".join(comments)
    messages = [
        {"role": "system", "content": "You are a customer success analyst summarizing feedback trends."},
        {"role": "user", "content": f"Summarize these customer service comments into an executive summary highlighting strengths, weaknesses, and areas for improvement:\n{joined_comments}"}
    ]
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    return response.choices[0].message.content

# Example
create_summary(df['customer_comment'].sample(5).tolist())

'Executive Summary:\n\nStrengths: Our product is receiving positive feedback for sound quality and user-friendliness, which indicates that our core features are meeting customer expectations. Our customer service efforts are also acknowledged as being helpful and responsive, highlighting the effectiveness of our customer interactions.\n\nWeaknesses: Customers are expressing dissatisfaction regarding the battery life of our products, with a noteworthy concern being about devices overheating quickly. These could potentially affect product reliability and customer safety. Issues with our pricing policy, specifically regarding price reductions shortly after purchase, are resulting in customer disappointment. Additionally, customers are still waiting for deliveries despite ordering two weeks prior, indicating a potential problem with our shipping or order management process.\n\nAreas for Improvement: We may need to revisit the engineering of our devices to address issues around battery life